For the Kernel Affine Projection algorithm. We initialize the discrete-time non-linear system through 

v_n = 1.1exp(-|v_n-1|) + u_n

with the desired output d_n being

d_n = v_n^2

For initialization, we initialize u_n from a zero-mean Guassian distribution  with standard deviation 0.25.

In [64]:
import numpy as np
import math as m

sigma = 0.25

np.random.normal(0,sigma)

-0.4840520115335717

For our kernel function, we will be using a modified Guassian kernel described as

k(ui,uj) = exp(-||ui - uj||/ b0)

for a varied optimized b0 from 0.1 to 1 in increments of 0.005

In [108]:
def KAPgaussianKernel(a,b):
    b0 = 0.25
    print("A",a)
    print("B",b)
    return np.exp(np.linalg.norm(a-b)**2 / b0)


In [109]:
def kerfineAffineProj(inp,out,size,mu,regularity,cohere,p1):
    # initialize dictionary
    dictionary = np.array(inp[1])
    dictionarySize = 1
    
    # now initialize the kernel with the dictionary
    ker = KAPgaussianKernel(dictionary,dictionary)
    
    # coherence of the dictionary
    cohDict = np.array(np.sqrt(ker))
    
    # initialize alpha
    alpha = np.array(out[0] / ker)
    
    # filtering loop
    
    for i in range(0,100):
        # grabbing u
        u = inp[i]
        pointProject = KAPgaussianKernel(dictionary,u)
        cohNew = KAPgaussianKernel(u,u)
        
        coherenceSpace = pointProject / np.sqrt(cohNew) * cohDict
        
        # if the new point is below coherence threshold
        if(np.amax(np.absolute(coherenceSpace)) < cohere):
            # add point to dictionary
            dictionary = np.append(dictionary,u)
            cohDict = np.append(cohDict, cohNew)
            dictionarySize = dictionarySize + 1
            
            # increase size of alpha to accomodate new size
            alpha = np.append(alpha,0)
        
        # compute H matrix
        # https://stackoverflow.com/questions/4647368/how-do-i-reverse-a-part-slice-of-a-list-in-python
        H = KAPgaussianKernel(dictionary,np.flipud(inp[i:max(i-size+3,2)]))
        D = np.flipud(out[i:max(i-size+1,1)])
        print(dictionary)
        print(H)
        print(alpha)
        # print(inp[i:max(i-size+3,2)])
        print(H @ alpha)
        E = D - H @ alpha
        
        # recalculating alpha
        alpha = np.append(alpha,alpha + mu * H.transpose() @ np.linalg.inv(H @ H.transpose() + regularity*np.identity(len(D))) @ E)
        print(alpha)
               

In [67]:
end = 500

un = np.random.normal(0,sigma,end)
v0 = 0.5
v1 = 1.1 * m.exp(v0) + un[0]



noise = np.random.normal(0,1,end)

d0 = v0 ** 2
d1 = v1 ** 2

v = np.array([v0,v1])
dground = np.array(d0,d1)
d = dground + noise[0:1]

# initializing the function with the first 500 samples
for i in range(2,end):
    v = np.append(v,1.1 * np.exp(- np.absolute(v[i-1])) + un[i])
    

In [110]:
print(kerfineAffineProj(v,d,2,0.003,0.0009,0.3,0.35))

A 2.0545525292327707
B 2.0545525292327707
A 2.0545525292327707
B 0.5
A 0.5
B 0.5
A 2.0545525292327707
B [2.05455253 0.5       ]
2.0545525292327707
15780.56288541872
0.7761842135140389


ValueError: matmul: Input operand 0 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [16]:
x = 0.5
2 * m.exp(x)



0.24499999999999997
